# Image Alignment and Stitching

<center><img src="figs/00_dip.jpeg" width=500px alt="default"/></center>


# Last Lecture

- 01: Image Formation Models (Ch. 2)
- 02: Image Operators and Transforms (Ch. 3)
- **03: Feature Detection, Matching, Segmentation (Ch. 7)**
- 04: Image Alignment and Stitching (Ch. 8)
- 05: 3D Reconstruction (Ch. 13)

We saw how to find features in the images:
- sometimes these features are meant to be "matched" for a downstream task,
- sometimes these features are interesting in themselves (boundaries of objects).

# Last Lecture

- 01: Image Formation Models (Ch. 2)
- 02: Image Operators and Transforms (Ch. 3)
- **03: Feature Detection, Matching, Segmentation (Ch. 7)**
- 04: Image Alignment and Stitching (Ch. 8)
- 05: 3D Reconstruction (Ch. 13)

We saw how to find features in the images:
- **sometimes these features are meant to be "matched" for a downstream task,**
- sometimes these features are interesting in themselves (boundaries of objects).

# This Lecture

- 01: Image Formation Models (Ch. 2)
- 02: Image Operators and Transforms (Ch. 3)
- 03: Feature Detection, Matching, Segmentation (Ch. 7)
- **04: Image Alignment and Stitching (Ch. 8)**
- 05: 3D Reconstruction (Ch. 13)

# Definitions

$\color{#EF5645}{\text{Matching features}}$ means to determine which features come from corresponding locations in different images.

$\color{#EF5645}{\text{Image alignment}}$ means to find the geometric transformation that can transform an image into another. This can be done using matched features.

$\color{#EF5645}{\text{Image stitching}}$ means "gluing" images together in a consistent way, for example to form a panorama. This can be done using image alignment.

<center><img src="figs/03_main2.png" width=400px alt="default"/></center>

# Image Alignment and Stitching

- **[Image Alignment](#sec-syllabus)**

- [Stitching](#sec-ece)

Assume a set of matching feature points $\left\{x_i, x_i'\right\}_{i=1}^N$ has been computed. Use them for alignment?


$\color{#EF5645}{\text{Image alignment}}$ means to find the geometric transformation that can transform an image into another. This can be done using matched features.

$\color{#6D7D33}{\text{Method}}$: Pairwise alignment from matching using least squares (LS).
- Given a set of $N$ matched feature points $\left\{x_i, x_i'\right\}_{i=1}^N$,
- Given a family of planar parametric transformation of the form $x' = f(x; p)$ with parameters $p$,
- Goal: Find the transformation parameters $p$ that minimize the LS problem:
$$ min_p E_{LS}(p) = \sum_{i=1}^N ||r_i||^2 = \sum_{i=1}^N ||f(x_i; p) - x_i'||^2.$$

$\color{#EF5645}{\text{Remark:}}$ $f(x_i; p)$ is the predicted location of the point matching $x_i$ in the second image.

# Role of Matched

- Least square approach, possibly reweighting each of the keypoint pair
- Pre-selecting good keypoints:
  - $\color{#EF5645}{\text{RANdom SAmple Consensus (RANSAC)}}$
  - $\color{#EF5645}{\text{Least Median of Squares (LMS)}}$
  - Both:
    - select at random a subset k of N matches.
    - compute hat p of transformations

$\color{#EF5645}{\text{Jacobian of the transformation}}$: Many geometric transformations, such as translation, similarity, affine, have a linear relationship between the amount of motion $\Delta x = x' - x$ and the unknown transformation parameters $p$:
$$\Delta x = x' - x = J(x) p,$$
where:
- $J(x) = \frac{\partial f}{\partial p}(x)$ is the Jacobian of the transformation $f$ with respect to $p$.

# Jacobians of Transformations

<center><img src="figs/04_jac.png" width=800px alt="default"/></center>


We can get the least square problems.

$\color{#6D7D33}{\text{Method}}$: Pairwise alignment from matching using least squares (LS).
- Given a set of $N$ matched feature points $\left\{x_i, x_i'\right\}_{i=1}^N$,
- Given a family of planar parametric transformation of the form $x' = f(x; p)$ with parameters $p$,
- Goal: Find the transformation parameters $p$ that minimize the LS problem:
$$ min_p E_{LS}(p) = \sum_{i=1}^N ||r_i||^2 = \sum_{i=1}^N ||f(x_i; p) - x_i'||^2.$$

